In [1]:
import numpy as np
import random
import time

In [2]:
'''
tables are integrated PDFs for initialization of maxwellians for electrons and ions
delgam in the input file sets the temperature in delta gamma, here it is converted 
between species by multiplying by mass.
!! gamma_table is gamma -1 table
'''
def init_maxw_table(delgam,pdf_sz):
    maxg        = (delgam)*20+1.
    gamma_table = np.linspace(0,0,pdf_sz)
    pdf_table   = np.linspace(0,0,pdf_sz)
    for i in range(pdf_sz):
        gamma_table[i]=(maxg-1.)/(pdf_sz-1)*i
    # a safer way of writing gamma*sqrt(gamma^2-1)*exp(-(gamma-1)/delgam)
    func=(gamma_table+1.)*np.sqrt(gamma_table*(gamma_table+2.))*np.exp(-(gamma_table)/delgam)  
    pdf_table[0]=0.
    for i in range(1,pdf_sz):
        pdf_table[i]=sum(func[0:i+1])
    #normalize pdf_table         
    pdf_table=pdf_table/pdf_table[pdf_sz-1]
    return gamma_table,pdf_table

In [3]:
# Maxwell distribution for the particles
def init_rel_maxwell(gamma0,pdf_sz,gamma_table,pdf_table):
    gamma0mag = np.abs(gamma0)
    beta_drift = np.sign(gamma0)*np.sqrt(1.-1./max(abs(gamma0),1.)**2)
    
    random.seed(time.time())    
    rannum = random.uniform(0,1)
    if (rannum==1.):
        rannum = random.uniform(0,1)
    gam=0. #gam = gamma-1., only kinetic energy to avoid underflows
    #choose gamma from the table
    for i in range(pdf_sz):               
        if(i==pdf_sz-1):
            gam=gamma_table[pdf_sz-1]
        if(rannum > pdf_table[i] and rannum < pdf_table[i+1]):                      
            gam=gamma_table[i]+(gamma_table[i+1]-gamma_table[i]) \
            /(pdf_table[i+1]-pdf_table[i])*(rannum-pdf_table[i])
            break
    
    pcosth = 2*random.uniform(0,1)-1
    pphi   = random.uniform(0,1)*2*np.pi
    psinth = np.sqrt(1-pcosth**2)   
    
    # v0t = beta
    v0t = np.sqrt((gam)*(gam+2.))/(1.+gam)     
    ut1 = v0t*psinth*np.cos(pphi)
    vt1 = v0t*psinth*np.sin(pphi)
    wt1 = v0t*pcosth
    
    # aka Zenitani 2015, u = gam*v, including c factor
    ptx = (1.+gam)*ut1
    pty = (1.+gam)*vt1
    ptz = (1.+gam)*wt1
    # flipping method aka Zenitani 2015, TABLE II
    X7  = random.uniform(0,1)
    if (-beta_drift*ut1>X7):
        ptx = -ptx
    # particles will have average velocity beta0 (with sign)    
    px1 = (ptx+beta_drift*(gam+1.))*gamma0mag #this will include the sign of gamma0
    py1 = pty
    pz1 = ptz
    
    gam1=np.sqrt(1+px1**2+py1**2+pz1**2)

    u=px1/gam1
    v=py1/gam1
    w=pz1/gam1
    return gam1,u,v,w

In [4]:
# delgam = k T/m c^2
def generate_maxwell_dist(num,delgam,pdf_sz,btheta,outform):
    gamma_table,pdf_table = init_maxw_table(delgam,pdf_sz)
    ux  = np.empty(num) # beta nor gamma*beta
    uy  = np.empty(num)
    uz  = np.empty(num)
    gam0= np.empty(num)
    for i in range(num):
        gam0[i],ux[i],uy[i],uz[i] = init_rel_maxwell(1.,pdf_sz,gamma_table,pdf_table)
    if (outform=='xyz'):
        return gam0, ux, uy, uz
    elif(outform=='polar'):
        btheta = 70*np.pi/180.
        bpa = ux*np.cos(btheta) +  uy*np.sin(btheta)
        u0  = -ux*np.sin(btheta) + uy*np.cos(btheta)
        bpr = np.sqrt(u0**2+uz**2)
        return gam0,bpa,bpr
    else:
        print ('select right outform: xyz or polar(|| and perp) \n')
        return 0

In [3]:
def prtl_distribution(gam1,gambins,gammin=-4,gammax=1):    
    gamma_list  = np.logspace(gammin,gammax,gambins) # gamma-1
    fe = np.empty(len(gamma_list))   
    for k in range(len(fe)):
        if k==0:
            fe[k] = sum(gam1<gamma_list[0]+1)/len(gam1)
        ar = (gam1>gamma_list[k-1]+1) & (gam1<gamma_list[k]+1)
        fe[k] = float(sum(ar))/len(gam1)
    return gamma_list, fe 